 # To scrap website

<b> Import necessary libraries</b>

In [50]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Libraries imported


<h3>Scrap the content</h3>

In [26]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


<h3> Replace '\n' by ' , '</h3>

In [27]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<h3>Group by Postalcode</h3>

In [28]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3> Shape </h3>

In [29]:
df.shape

(103, 3)

# Give Longititude and Latitude to neighborhood

 <h3> Downloaded Geospatial data because geocoder is not working</h3>
   

In [57]:
!wget -q -O 'geo_data.csv'  http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


<h3>Load the data</h3>

In [58]:
df_geo_coor = pd.read_csv("geo_data.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3> merge df and df_geo_coor</h3>

In [59]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Explore and cluster neighborhood in Torronto

<b> 1) Getting latitude and longitude</b>

In [60]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinate of Toronto city are {}, {}.'.format(latitude, longitude))


The coordinate of Toronto city are 43.653963, -79.387207.


<b> 2) Map neighborhood of Torronto </b>

In [63]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9)
map_toronto

<b> 3) Adding markers</b>

In [64]:
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

<b> 3) map torronto city only</b>

<h4> working with borough containing torronto </h4>

In [65]:
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [66]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc


<b>4)Define Foursquare credential</b>
   Credential is not mentioned as it is public repository on  github

In [67]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

<b>5) explore first neighborhood</b>

In [68]:
neighborhood_name = df_toronto_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


In [69]:
neighborhood_latitude = df_toronto_denc.loc[0, 'Latitude'] 
neighborhood_longitude = df_toronto_denc.loc[0, 'Longitude'] 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [70]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=&ll=&radius=&limit='.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [71]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [72]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll=,&radius=&limit='.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [86]:
toronto_denc_venues = getNearbyVenues(names=df_toronto_denc['Neighborhood'],
                                   latitudes=df_toronto_denc['Latitude'],
                                   longitudes=df_toronto_denc['Longitude']
                                  )
toronto_denc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
2,The Beaches,43.676357,-79.293031,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
3,The Beaches,43.676357,-79.293031,Cafe Plenty,43.654571,-79.389450,Café
4,The Beaches,43.676357,-79.293031,Poke Guys,43.654895,-79.385052,Poke Place


In [87]:
toronto_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",80,80,80,80,80,80
Berczy Park,80,80,80,80,80,80
"Brockton, Exhibition Place, Parkdale Village",80,80,80,80,80,80
Business Reply Mail Processing Centre 969 Eastern,80,80,80,80,80,80
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",80,80,80,80,80,80
"Cabbagetown, St. James Town",80,80,80,80,80,80
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",80,80,80,80,80,80
Christie,80,80,80,80,80,80


In [88]:
print('{} uniques categories.'.format(len(toronto_denc_venues['Venue Category'].unique())))

57 uniques categories.


<b> 7) Analysis of each neighborhood</b>

In [89]:
toronto_denc_onehot = pd.get_dummies(toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = toronto_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()


,Vegetarian / Vegan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Breakfast Spot,Bubble Tea Shop,...,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Toy / Game Store,University
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
toronto_denc_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
toronto_denc_grouped.head()

,Neighborhood,Vegetarian / Vegan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Breakfast Spot,...,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Toy / Game Store,University
0,"Adelaide, King, Richmond",0.025,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125,0.025,0.0375,...,0.0125,0.025,0.0125,0.0125,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125
1,Berczy Park,0.025,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125,0.025,0.0375,...,0.0125,0.025,0.0125,0.0125,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125
2,"Brockton, Exhibition Place, Parkdale Village",0.025,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125,0.025,0.0375,...,0.0125,0.025,0.0125,0.0125,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125
3,Business Reply Mail Processing Centre 969 Eastern,0.025,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125,0.025,0.0375,...,0.0125,0.025,0.0125,0.0125,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.025,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125,0.025,0.0375,...,0.0125,0.025,0.0125,0.0125,0.0125,0.0375,0.0125,0.0125,0.0125,0.0125


In [106]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted1.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
1,Berczy Park,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop


<b>8) Clustering Neighborhood</b>

In [104]:
kclusters = 5

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [107]:
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop


In [94]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b> 9) Examine cluster

<b> cluster 1</b>

In [95]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
1,East Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
2,East Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
3,East Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
4,Central Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
5,Central Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
6,Central Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
7,Central Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
8,Central Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
9,Central Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Sushi Restaurant,Breakfast Spot,Chinese Restaurant,Hotel,Bar,Bubble Tea Shop
